In [1]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [10]:
from dotenv import load_dotenv
load_dotenv()
import os

In [11]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")

In [4]:
#Arxiv Tool for research papers
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv,description="Query arxiv papers")


In [5]:
arxiv.invoke("Attention is all you need")

"Published: 2024-07-22\nTitle: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models\nAuthors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini\nSummary: The inference demand for LLMs has skyrocketed in recent months, and serving\nmodels with low latencies remains challenging due to the quadratic input length\ncomplexity of the attention layers. In this work, we investigate the effect of\ndropping MLP and attention layers at inference time o"

In [6]:
#Wikipedia tool for wikipedia articles
api_wrapper_wikipedia=WikipediaAPIWrapper(top_k_results=2,doc_content_chars_max=500)
wikipedia=WikipediaQueryRun(api_wrapper=api_wrapper_wikipedia,description="Query Wikipedia articles")

In [13]:
#Tavily tool for web-search
from langchain_community.tools.tavily_search import TavilySearchResults
tavily=TavilySearchResults(api_key='TAVILY_API_KEY',description="Query web-search results")
tavily.invoke("tell me recent US tariff news")

[{'url': 'https://finance.yahoo.com/news/live/trump-tariffs-live-updates-possible-tech-auto-exemptions-in-focus-as-trump-team-sows-confusion-191201321.html',
  'content': "Sun, April 13, 2025 at 12:51 PM UTC\nThe tariff uncertainty isn't getting any better in markets\nAs our colleague Josh Schafer writes, as of April 3, the Yale Budget Lab had estimated the effective US tariff rate was 22.5%. Now, it estimates the US effective tariff rate is 27%.\nAnd uncertainty in what tariffs may or may not be applied — late Friday, the Trump administration exempted many consumer electronics including smartphones and computers — is causing consternation on Wall Street: [...] Sat, April 12, 2025 at 12:04 PM UTC\nGrace O'Donnell\nEuro strengthens, dollar loses luster amid tariff wars\nThe euro has emerged as a surprise winner of the recent tariff-induced market turmoil, Reuters reports.\nFollowing the November 2024 election, the United States was seen as the exceptional market and economy. Now, Europe

In [14]:
#Putting tools in one list
tools=[arxiv,wikipedia,tavily]

In [15]:
#Introducing LLM
from langchain_groq import ChatGroq
llm=ChatGroq(model="llama-3.3-70b-versatile")

In [16]:
llm.invoke("Hello")

AIMessage(content='Hello. How can I help you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46, 'completion_time': 0.036363636, 'prompt_time': 0.001628756, 'queue_time': 0.058278632999999996, 'total_time': 0.037992392}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'finish_reason': 'stop', 'logprobs': None}, id='run-203f5b05-947c-4c59-b62c-88f375bc954f-0', usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46})

In [17]:
llm_with_tools=llm.bind_tools(tools)

In [18]:
llm_with_tools.invoke("Tell me recent research on AI")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_fvk0', 'function': {'arguments': '{"query": "recent AI research"}', 'name': 'arxiv'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 339, 'total_tokens': 356, 'completion_time': 0.069001455, 'prompt_time': 0.022160778, 'queue_time': 0.055866535, 'total_time': 0.091162233}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-adc6943d-95c5-4112-82ba-994b7469f576-0', tool_calls=[{'name': 'arxiv', 'args': {'query': 'recent AI research'}, 'id': 'call_fvk0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 339, 'output_tokens': 17, 'total_tokens': 356})

## Workflow

In [19]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage #Human msg or AI msg
from typing import Annotated
from langgraph.graph.message import add_messages #Reducers in Langgraph

In [20]:
class State(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]

In [21]:
from IPython.display import Image, display
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

In [22]:
#Node definition
def assistant(state:State):
    return {"messages":[llm_with_tools.invoke(state["messages"])]}


In [23]:
builder=StateGraph(State)
builder.add_node("Assistant",assistant)
builder.add_node("tools",ToolNode(tools))
builder.add_edge(START,"Assistant")
builder.add_conditional_edges("Assistant",tools_condition) # Conditional tool call, if tool needed edge to tool else edge to END
builder.add_edge("tools","Assistant") #Tool call edge to assistant
builder.add_edge("Assistant",END)

In [24]:
graph=builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)

In [25]:
messages=graph.invoke({"messages":"1706.03762"})
for i in messages['messages']:
    i.pretty_print()

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

In [26]:
messages=graph.invoke({"messages":"Hello, what's up please tell me about Indus Valley Civilization"})
for i in messages['messages']:
    i.pretty_print()

================================ Human Message =================================

Hello, what's up please tell me about Indus Valley Civilization
================================== Ai Message ==================================
Tool Calls:
  wikipedia (call_rn4q)
 Call ID: call_rn4q
  Args:
    query: Indus Valley Civilization
================================= Tool Message =================================
Name: wikipedia

Page: Indus Valley Civilisation
Summary: The Indus Valley Civilisation (IVC), also known as the Indus Civilisation, was a Bronze Age civilisation in the northwestern regions of South Asia, lasting from 3300 BCE to 1300 BCE, and in its mature form from 2600 BCE to 1900 BCE. Together with ancient Egypt and Mesopotamia, it was one of three early civilisations of the Near East and South Asia, and of the three, the most widespread, its sites spanning an area including much of Pakistan, northwestern I
================================== Ai Message ===========================